<a id="top"></a>
# Lesson 03: Astroseismology and Other "Spurious" Signals
***
## Learning Goals
By the end of this tutorial, you will be able to:
- List possible non-planetary sources of periodic signals in light curves, like binary and variable stars.
- Subtract a non-planetary signal from a periodogram to reveal the signal of a transiting exoplanet.
- List possible sources of false transit positives, including the effects of TESS’s orbit and detector noise.

## Introduction

Exoplanetary science is one of the newest and most rapidly growing fields of astrophysics, as people try to find and characterize planets beyond our solar system. One of the most common ways to detect the existence of an exoplanet is via the resulting, periodic dip in the light curve of its star as the planet passes in front. However, there are some other periodic signals that affect light curves that can appear similar to and could be mistaken for an exoplanet transit, such as transiting binaries. We'll take a look at both of these cases and how to properly interpret those signals.

This Notebook assumes some familiarity with [binary stars](https://www.atnf.csiro.au/outreach/education/senior/astrophysics/binary_intro.html), [exoplanets and their transits](https://avanderburg.github.io/tutorial/tutorial.html), and [light curves](https://imagine.gsfc.nasa.gov/science/toolbox/timing1.html).

### Why Would a Binary System Light Curve Look Similar to an Exoplanet Transit?
There are several non-planetary sources, such as binary stars or variable stars (cepheids, RR Lyrae, etc.), that periodcally change brightness, and can appear similar to an exoplanet transit in front of a star.

Let's look at the light curve of a binary star and compare it to that of a transiting planet.

![Astroseismology Diagram.png](light_curves.png)

Binary stars, by definition, orbit each other. If this orbit is aligned along the line-of-sight of the observer, periodic dimming in brightness will occur; critically, this can resemble an exoplanet transit. This resemblance between binary star eclipses and exoplanet transits is particularly strong when the two binary stars are of similar size and brightness, as then the 'dips' will be closer in size and look more like a repeated planet signal.

## Imports

We'll be using the "typical" packages for the webinar. Of note:

- `astroquery.mast` to query for Observations
- `s3fs` to access cloud data
- `lightkurve` to analyze light curves

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
import s3fs
import warnings

from astropy.table import Table, Column
from astroquery.mast import Mast
from astroquery.mast import Observations
from astropy.io import fits

# Enable the cloud filesystem
fs = s3fs.S3FileSystem(anon=True)
Observations.enable_cloud_dataset()

## Retrieving a Light Curve

We'll start our query, as we have previously, by picking a target, filtering for files, then turning them into cloud URIs. Today's target is `V* V839 Cep`, which is a known binary star system. To speed up the search, we'll use its corresponding TIC ID. We'll also limit results to sectors 15, 16, and 17, both for future consistency and to further optimize this query.

In [ ]:
# Retrieving the data directly using astroquery
obs = Observations.query_criteria(target_name = "375422201"
                                  , obs_collection="TESS"
                                  , dataproduct_type="timeseries"
                                  , sequence_number=[15, 16, 17])

# Pick which products we want to retrieve
data_prod = Observations.get_product_list(obs)
filt_prod = Observations.filter_products(data_prod, description="Light curves")

c_uris = Observations.get_cloud_uris(filt_prod)
c_uris

### Loading into the Lightkurve Package

In this lesson, we'll use a new, open-source tool: [Lightkurve](https://docs.lightkurve.org). This software was specifically desgined for use with data from the Kepler and TESS missions. It contains a few handy features, like:
- Automatic, flux-normalized stitching between sectors
- Built-in phase-folded plotting
- Convenient [fourier transforms](https://www.youtube.com/watch?v=spUNpyF58BY) to create periodograms and determine orbital periods

Lightkurve is a wonderful tool. Unfortunately, it hasn't yet been updated to work in a cloud environment so our approach here will be a little clunky. We first initialize a lightcurve object, passing in the cloud file. Then, we can create a light curve collection of each subsequent file, and stitch them together.

Let's start by opening and plotting just the first light curve.

In [ ]:
# Initialize a TESS lightCurve in lightKurve
with fs.open(c_uris[0], "rb") as f:
    with fits.open(f, "readonly") as fts:
        lc1 = lk.TessLightCurveFile(fts)

# create a plot
lc1.plot()

Great. This is somewhere around 27 days of data, so we know we've plotted the first sector. At this point, we can also notice the characteristic dips of an eclipsing binary: there are two, equally-spaced peaks of varying depths. 

There is also something interesting happening at the top of this light curve. It's not flat, and it looks like we might even have a teriary transit? Let's add data from other sectors before we analyze further.

In [ ]:
# Loop through the remaining lightcurves and stitch them
for lc in c_uris[1:]:
    with fs.open(lc, "rb") as f:
        with fits.open(f, "readonly") as fts:
            new_lc = lk.TessLightCurveFile(fts)
            lc1 = lk.LightCurveCollection((lc1, new_lc)).stitch()

# Create the plot of the new, combined light curve
lc1.plot()

The patterns we noticed in the previous sector hold true when we add more data. That's good news! Something interesting is happening in this star system.

## Creating a Periodogram

A periodogram relies on [fourier transforms](https://www.youtube.com/watch?v=spUNpyF58BY) to turn our signal into its associated frequencies. The mathematics underlying this transformation is fascinating and applicable to a wide range of fields, from quantum mechanics to mixing music.

In `lightkurve`'s implementation, we need to provide the range of frequencies in days. We also need to specify a method for computing the transformation; in this case, we'll use the box-least square method from astropy.

![bls](https://docs.astropy.org/en/stable/_images/bls-1.png)

This model should work well under these conditions. Let's try running it now. We'll save the period that is most likely, for later use.

In [ ]:
# Using BLS to turn it into a periodogram
period = np.linspace(1, 20, 10000)
bls = lc1.to_periodogram(method='bls', period=period);
bls.plot();

# Calculating most likely parameters
binary_period = bls.period_at_max_power
binary_period

The "BLS Power" represents the strength of the associated periods. A higher peak means that there is a stronger signal at that period of oscillation. There are also harmonics: integer multiples of the true period that show up in our graph.

In this case, it looks like the highest period is 5 days, with harmonics at 10 (harmonic 2) and 2.5 (harmonic 1/2). To judge how well these values fit, we can [create a phase-folded light curve](https://www.youtube.com/watch?v=UyyNKuoHYcA). "Folding" the light curve is a natural way to look at periodic changes in brightness, since an accurate period will cause patterns to line up at each point in the phase.

Let's try creating a folded light curve to see how well our value matches.

In [ ]:
lc1.fold(period=binary_period).scatter()

Oh no! Something is not quite right here. There are two deep transits of slightly similar depths, but they're nearly on top of each other.

### Exercise: What's the real period?

If we'd picked the correct value, we'd expect to see two separate transits: one for the primary eclipse, and another for the secondary. Thinking about the orbits of binary stars, what might be wrong with our calculated period? Can you find the right period and create a new phase-folded curve?

In [ ]:
## type your answer here!

# true_period = binary_period
# lc1.fold(true_period).scatter()

## Masking out the Stellar Transits

There's definitely an even more interesting signal hiding in this data. To detect it, we'll need to first remove the signal from our binary stars. A good first approach to this might be to remove the data where it dips below a certain value.

In [ ]:
transiting = lc1['flux'] < 0.94
lc1[transiting].scatter()

Unfortunately, this won't quite work. There's still the area of these transits between 0.94 and 1 that will contribute to the periodogram. Instead, let's work to estimate the duration of these transits, which will allow us to create a better mask for them.

To start, let's get the times for the first and second transit.

In [ ]:
# convert time and flux to arrays, so that they're easier to work with
fluxes = lc1.flux.value
time_series = lc1.time.value

# identify the locations of the transits
transits = time_series[np.where(fluxes < 0.95)]

# identify the very first transit
first = transits[transits < 1712]

# identify the second transit
second = transits[np.bitwise_and(transits<1718, transits>1716)]

Now, since we have rough estimates for when these transits occured, we can figure out the time (the "middle" of the transit) and duration (how long they last). Our duration will be a bit of an underestimate, so we'll need to correct for that later.

In [ ]:
# Get the mean time for the first transit, calculate the duration
t_time1 = np.mean(first)
t_duration1 = first[-1] - first[0]

# Get the mean time for the first transit, calculate the duration
t_time2 = np.mean(second)
t_duration2 = second[-1] - second[0]

With these values calculated, we can ask `lightkurve` to create a transit mask. This will mark of the locations that we belive contain a transit. We know that our duration is an underestimate, so let's add a fudge factor of 10 to make sure we get all of the data.

For clarity, we'll plot both the transit we intend to remove and the resulting "cleaned" plot.

In [ ]:
# Create the mask for the larger transit, multiplying the duration by ten
mask1 = lc1.create_transit_mask(transit_time=t_time1, period=true_period, duration=t_duration1*10)

# show the masked data that we'll remove
lc1[mask1].scatter()
plt.title("Transits to be removed")

# plot the cleaned data
lc1[~mask1].scatter()
plt.title("Cleaned flux")

Good! Our factor of 10 seems about right for this graph: some transits have additional signal on both sides, but we haven't removed too much data.

Let's do the same for the secondary eclipse. Since we're capturing more of the transit in our intital estimate, our fudge factor will be a little bit lower.

In [ ]:
# Create the mask for the smaller transit, multiplying the duration by three
mask2 = lc1.create_transit_mask(transit_time=t_time2, period=true_period, duration=t_duration2*3)

# plot the transits we're removing
lc1[mask2].scatter()
plt.title("Transits to be removed")

# plot the cleaned data
lc1[~mask2].scatter()
plt.title("Cleaned flux")

Now that we've plotted each transit removed, we need to combine our results. Since we want to remove data anywhere there's a mask, we should use a bitwise `OR`.

In [ ]:
# combine our transit models
combined_transit_mask = np.bitwise_or(mask1, mask2)

# plot the flux with both transits removed
no_binary = lc1[~combined_transit_mask]
no_binary.scatter()

Excellent. With the binary star transits removed, it's clear that we still have some signal remaining. Let's analyze this as well.

## Looking for Additional Signals

To figure out the period of the other signal, we'll once again use a periodogram. This time though, we'll run it on our cleaned data.

In [ ]:
# Using BLS to turn it into a periodogram
periods = np.linspace(1, 20, 10000)
bls = no_binary.to_periodogram(method='bls', period=periods, frequency_factor=500);
bls.plot();

# Calculating most likely parameters if there is an exoplanet from periodogram
new_period = bls.period_at_max_power
new_period

Our new period is 4.07 days. Let's see how this looks when we fold it. For clarity, we'll add an epoch time of `1713.2` to better center the folded light curve.

In [ ]:
no_binary.fold(period=new_period, epoch_time=1713.2).scatter()

The shape of the transit here is certainly suggestive. In fact, this was indentified as a candidate multiple system in [Eisner et al. 2020](https://doi.org/10.1093/mnras/staa3739). However, there has been no follow-up on this target since the paper was published, so the true identity of this object remains a mystery.

We haven't discussed all of the features of this folded light curve yet. If you want to take this anaylsis a step further, you might investigate the variability of these stars. It's clear that there is some sort of intrinsic sinusodial pattern present in the data...

## Next time, on MAST Summer Webinar...

We'll look at transient phenomena that are observed by TESS. Get ready to hunt for supernovae!

## Additional Resources

Here are some additional resources to check out if you are interested in diving further in:

- [Lightkurve documentation](https://docs.lightkurve.org)
- [TESS Eclipsing Binaries](https://archive.stsci.edu/hlsp/tess-ebs)
- [List of Kepler Binaries](https://archive.stsci.edu/kepler/eclipsing_binaries.html)
- [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html): a queryable database of exoplanets
- [ExoMAST](https://exo.mast.stsci.edu/), MAST's database of exoplanets

## About this Notebook

**Author(s):** Thomas Dutkiewicz, Larom Segev <br>
**Keyword(s):** TIKE, Cloud archive, Astroseismology, Binary stars, Exoplanet transits <br>
**Last Updated:** June 2024 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 